In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("..")

from diabetes_prediction.utils.data.dataloader import *

In [ ]:
data_id = 'family'
meta = load_metadata(data_id, overwrite=True)
meta.head()

* _get_summary                 | 0.84s (0.01m)
